In [81]:
class Params:
	"""
	Parameters class.

	This file centralizes anything that can be 
	parametrized in the code.
	"""

	raw_data = '../data/raw/'
	processed_data = '../data/processed/'

	log_name = '../log/dump.log'

	# if this is set to True, then all the nodes will be automatically 
	# considered not up-to-date and will be rerun.
	rerun = True

	today = datetime.now().strftime("%Y-%m-%d")

	url_abc = 'https://www.superabc.com.br/'
	chrome_path = 'C:/Users/pedro/selenium/chromedriver.exe'
	path_base = 'C:/Users/pedro/Ironhack_DAFT/Labs/week4_remote/Data Pipeline/web_scraping_super_abc/'

	#path's to use in data preparation
	path_data_raw = path_base + 'data/raw/'
	sub_cat_processed = path_data_raw + 'sub_cat.csv'
	product_links =  path_data_raw +'product_links.csv'
	last_update_links = path_data_raw + 'last_update.txt'
	days_to_update_prep = 15

	#path's to use in data processed
	path_data_processed = path_base + 'data/processed/'
	links_processed = path_data_processed + 'links_worked.csv'
	data_csv = path_data_processed + today + '.csv'

In [82]:
import logging
import csv
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
from multiprocessing import Pool
from datetime import datetime
import os

logger = logging.getLogger('nodes.data_preparation')


def abrir_site():
    """
    Função que leva a home do site, como o o selenium não possuí um cache,
    a cada acesso é necessário selecionar a cidade em uma lista, bem como fechar um popup.
    @return: driver já com o acesso a home
    """
    # define url a ser utilizada
    url = Params.url_abc
    # define o chrome como webdriver a ser utilizado pelo selenium
    driver = webdriver.Chrome(executable_path=Params.chrome_path)
    #maximiza a janela
    driver.maximize_window()
    #efetua um get na url definida anteriormente
    driver.get(url)
    #seleciona a cidade na lista que aparece no site
    Select(driver.find_element_by_xpath('//*[@id="preHomeCidade"]')).select_by_value("Divinópolis")
    #efetua novo get após a seleção da cidade
    driver.get(url)
    #cria elemento a ser fechado pelo popup e clicka nele
    elem = driver.find_element_by_xpath('//*[@id="popupCorona"]/span')
    elem.click()
    #retorna o driver
    return driver

def get_cat_II(cat_I : str):
    """
    Busca os links das sub categorias para cada categoria criada anteriormente, e retorna em uma lista
    @param cat_I:link com a categoria mãe
    @return:lista com links das sub categorias, encontradas na categoria mãe do informada
    """
    # através da função abrir_site() gera o driver já acessado na home
    driver = abrir_site()
    # efetua um get na categoria mãe passada anteriormente
    driver.get(cat_I)
    #aguarda o get busca a opção que expande as sub categorias e clicka nele
    elem = driver.find_element_by_xpath('/html/body/div[2]/div/main/div[1]/div[2]/div/div/div[1]/button')
    elem.click()
    time.sleep(1)
    html = driver.page_source
    #retorna os links
    return html


In [83]:
html = get_cat_II('https://www.superabc.com.br/bazar')

In [108]:
soup = BeautifulSoup(html, 'lxml')
soup = soup.find_all('ul', {'style':'display: block; overflow: hidden;'})
soup = [item.find_all('li') for item in soup]
soup =  [item for li in soup for item in li]
links = [item.find('a')['href'].replace('20','50') for item in soup]
base = list(set(['/'.join(item.split('/')[0:4])+'?PS=50' for item in links]))
base2 = list(set(['/'.join(item.split('/')[0:5])+'?PS=50' for item in links]))
links = links+base
links = links+base2

In [110]:
x=[['https://www.superabc.com.br/mercearia/bomboniere/bombons-e-chocolates?PS=50', 'https://www.superabc.com.br/mercearia/bomboniere/guloseimas?PS=50', 'https://www.superabc.com.br/mercearia/bomboniere/doces-diversos?PS=50', 'https://www.superabc.com.br/mercearia/funcionais-diet-e-light/integrais?PS=50', 'https://www.superabc.com.br/mercearia/funcionais-diet-e-light/soja?PS=50', 'https://www.superabc.com.br/mercearia/funcionais-diet-e-light/diet?PS=50', 'https://www.superabc.com.br/mercearia/funcionais-diet-e-light/sem-gluten?PS=50', 'https://www.superabc.com.br/mercearia/funcionais-diet-e-light/sem-lactose?PS=50', 'https://www.superabc.com.br/mercearia/funcionais-diet-e-light/light?PS=50', 'https://www.superabc.com.br/mercearia/funcionais-diet-e-light/organicos?PS=50', 'https://www.superabc.com.br/mercearia/funcionais-diet-e-light/cereais?PS=50', 'https://www.superabc.com.br/mercearia/matinais/complemento-alimentar?PS=50', 'https://www.superabc.com.br/mercearia/matinais/achocolatados?PS=50', 'https://www.superabc.com.br/mercearia/matinais/bebidas-lacteas?PS=50', 'https://www.superabc.com.br/mercearia/matinais/cafes?PS=50', 'https://www.superabc.com.br/mercearia/matinais/chas?PS=50', 'https://www.superabc.com.br/mercearia/cereais-oleos-e-azeites/oleos-vegetais?PS=50', 'https://www.superabc.com.br/mercearia/cereais-oleos-e-azeites/azeites?PS=50', 'https://www.superabc.com.br/mercearia/cereais-oleos-e-azeites/acucar?PS=50', 'https://www.superabc.com.br/mercearia/cereais-oleos-e-azeites/arroz?PS=50', 'https://www.superabc.com.br/mercearia/cereais-oleos-e-azeites/feijao?PS=50', 'https://www.superabc.com.br/mercearia/cereais-oleos-e-azeites/cesta-basica?PS=50', 'https://www.superabc.com.br/mercearia/conservas-e-enlatados/carnes-e-peixes?PS=50', 'https://www.superabc.com.br/mercearia/conservas-e-enlatados/atomatados?PS=50', 'https://www.superabc.com.br/mercearia/conservas-e-enlatados/outros?PS=50', 'https://www.superabc.com.br/mercearia/conservas-e-enlatados/condimento-e-temperos?PS=50', 'https://www.superabc.com.br/mercearia/doces-e-sobremesa/embalados?PS=50', 'https://www.superabc.com.br/mercearia/doces-e-sobremesa/geleias?PS=50', 'https://www.superabc.com.br/mercearia/doces-e-sobremesa/misturas?PS=50', 'https://www.superabc.com.br/mercearia/graos-e-farinaceos/farinaceos?PS=50', 'https://www.superabc.com.br/mercearia/graos-e-farinaceos/graos?PS=50', 'https://www.superabc.com.br/mercearia/graos-e-farinaceos/polvilho?PS=50', 'https://www.superabc.com.br/mercearia/graos-e-farinaceos/sopas-e-caldos?PS=50', 'https://www.superabc.com.br/mercearia/graos-e-farinaceos/massas?PS=50', 'https://www.superabc.com.br/mercearia?PS=50', 'https://www.superabc.com.br/mercearia/funcionais-diet-e-light?PS=50', 'https://www.superabc.com.br/mercearia/graos-e-farinaceos?PS=50', 'https://www.superabc.com.br/mercearia/doces-e-sobremesa?PS=50', 'https://www.superabc.com.br/mercearia/matinais?PS=50', 'https://www.superabc.com.br/mercearia/conservas-e-enlatados?PS=50', 'https://www.superabc.com.br/mercearia/bomboniere?PS=50', 'https://www.superabc.com.br/mercearia/cereais-oleos-e-azeites?PS=50'], ['https://www.superabc.com.br/bebidas/cervejas/pilsen?PS=50', 'https://www.superabc.com.br/bebidas/cervejas/premium?PS=50', 'https://www.superabc.com.br/bebidas/cervejas/artesanais?PS=50', 'https://www.superabc.com.br/bebidas/cervejas/sem-alcool?PS=50', 'https://www.superabc.com.br/bebidas/sucos-refrescos-e-refrigerantes/refrigerantes?PS=50', 'https://www.superabc.com.br/bebidas/sucos-refrescos-e-refrigerantes/sucos-e-refrescos?PS=50', 'https://www.superabc.com.br/bebidas/whisky/nacional?PS=50', 'https://www.superabc.com.br/bebidas/whisky/importados?PS=50', 'https://www.superabc.com.br/bebidas/destilados/aguardente?PS=50', 'https://www.superabc.com.br/bebidas/destilados/vodka?PS=50', 'https://www.superabc.com.br/bebidas/destilados/ice?PS=50', 'https://www.superabc.com.br/bebidas/destilados/conhaque?PS=50', 'https://www.superabc.com.br/bebidas/destilados/aperitivos?PS=50', 'https://www.superabc.com.br/bebidas/destilados/outros?PS=50', 'https://www.superabc.com.br/bebidas/energeticos-e-isotonicos/energeticos?PS=50', 'https://www.superabc.com.br/bebidas/aguas-e-chas/agua-mineral?PS=50', 'https://www.superabc.com.br/bebidas/aguas-e-chas/agua-de-coco?PS=50', 'https://www.superabc.com.br/bebidas/aguas-e-chas/chas-prontos?PS=50', 'https://www.superabc.com.br/bebidas?PS=50', 'https://www.superabc.com.br/bebidas/energeticos-e-isotonicos?PS=50', 'https://www.superabc.com.br/bebidas/sucos-refrescos-e-refrigerantes?PS=50', 'https://www.superabc.com.br/bebidas/cervejas?PS=50', 'https://www.superabc.com.br/bebidas/destilados?PS=50', 'https://www.superabc.com.br/bebidas/aguas-e-chas?PS=50', 'https://www.superabc.com.br/bebidas/whisky?PS=50'], ['https://www.superabc.com.br/carnes-e-pescados/pescados/peixe-de-agua-salgada?PS=50', 'https://www.superabc.com.br/carnes-e-pescados/pescados/peixe-de-agua-doce?PS=50', 'https://www.superabc.com.br/carnes-e-pescados/pescados/frutos-do-mar?PS=50', 'https://www.superabc.com.br/carnes-e-pescados/embutidos/linguicas?PS=50', 'https://www.superabc.com.br/carnes-e-pescados/embutidos/defumados-e-salgados?PS=50', 'https://www.superabc.com.br/carnes-e-pescados/embutidos/embutidos?PS=50', 'https://www.superabc.com.br/carnes-e-pescados/embutidos/presuntaria?PS=50', 'https://www.superabc.com.br/carnes-e-pescados?PS=50', 'https://www.superabc.com.br/carnes-e-pescados/embutidos?PS=50', 'https://www.superabc.com.br/carnes-e-pescados/pescados?PS=50'], ['https://www.superabc.com.br/congelados-e-resfriados/sucos-polpas-e-acais/sucos?PS=50', 'https://www.superabc.com.br/congelados-e-resfriados/petiscos-e-empanados/quibes?PS=50', 'https://www.superabc.com.br/congelados-e-resfriados/sobremesas-e-sorvetes/sorvetes-e-picoles?PS=50', 'https://www.superabc.com.br/congelados-e-resfriados?PS=50', 'https://www.superabc.com.br/congelados-e-resfriados/sucos-polpas-e-acais?PS=50', 'https://www.superabc.com.br/congelados-e-resfriados/petiscos-e-empanados?PS=50', 'https://www.superabc.com.br/congelados-e-resfriados/sobremesas-e-sorvetes?PS=50'], [], ['https://www.superabc.com.br/padaria/paes-industrializados/paes-industrializados?PS=50', 'https://www.superabc.com.br/padaria/paes-industrializados/bolos?PS=50', 'https://www.superabc.com.br/padaria?PS=50', 'https://www.superabc.com.br/padaria/paes-industrializados?PS=50'], ['https://www.superabc.com.br/hortifruti/processados-e-selecionados/processados?PS=50', 'https://www.superabc.com.br/hortifruti/processados-e-selecionados/selecionados?PS=50', 'https://www.superabc.com.br/hortifruti/processados-e-selecionados/exoticos?PS=50', 'https://www.superabc.com.br/hortifruti?PS=50', 'https://www.superabc.com.br/hortifruti/processados-e-selecionados?PS=50'], ['https://www.superabc.com.br/higiene-e-beleza/higiene-pessoal/desodorantes?PS=50', 'https://www.superabc.com.br/higiene-e-beleza/higiene-pessoal/produtos-para-pele?PS=50', 'https://www.superabc.com.br/higiene-e-beleza/higiene-pessoal/sabonetes?PS=50', 'https://www.superabc.com.br/higiene-e-beleza/higiene-pessoal/absorventes?PS=50', 'https://www.superabc.com.br/higiene-e-beleza/higiene-pessoal/para-barbear?PS=50', 'https://www.superabc.com.br/higiene-e-beleza/higiene-pessoal/acessorios?PS=50', 'https://www.superabc.com.br/higiene-e-beleza/maquiagem/esmaltes?PS=50', 'https://www.superabc.com.br/higiene-e-beleza/maquiagem/batons?PS=50', 'https://www.superabc.com.br/higiene-e-beleza/maquiagem/especiais?PS=50', 'https://www.superabc.com.br/higiene-e-beleza/infantil/pele?PS=50', 'https://www.superabc.com.br/higiene-e-beleza/infantil/cabelo?PS=50', 'https://www.superabc.com.br/higiene-e-beleza/infantil/higiene?PS=50', 'https://www.superabc.com.br/higiene-e-beleza/infantil/acessorios?PS=50', 'https://www.superabc.com.br/higiene-e-beleza/higiene-oral/escovas?PS=50', 'https://www.superabc.com.br/higiene-e-beleza/higiene-oral/fio-dental?PS=50', 'https://www.superabc.com.br/higiene-e-beleza/higiene-oral/cremes-e-gel?PS=50', 'https://www.superabc.com.br/higiene-e-beleza/higiene-oral/antisseptico?PS=50', 'https://www.superabc.com.br/higiene-e-beleza/saude-e-bem-estar/saude?PS=50', 'https://www.superabc.com.br/higiene-e-beleza/saude-e-bem-estar/preservativos?PS=50', 'https://www.superabc.com.br/higiene-e-beleza?PS=50', 'https://www.superabc.com.br/higiene-e-beleza/higiene-pessoal?PS=50', 'https://www.superabc.com.br/higiene-e-beleza/saude-e-bem-estar?PS=50', 'https://www.superabc.com.br/higiene-e-beleza/infantil?PS=50', 'https://www.superabc.com.br/higiene-e-beleza/higiene-oral?PS=50', 'https://www.superabc.com.br/higiene-e-beleza/maquiagem?PS=50'], ['https://www.superabc.com.br/limpeza/ceras-e-desinfetantes/desinfetantes?PS=50', 'https://www.superabc.com.br/limpeza/alvejantes-e-amaciantes/amaciantes?PS=50', 'https://www.superabc.com.br/limpeza/alvejantes-e-amaciantes/tinturas-e-alvejantes?PS=50', 'https://www.superabc.com.br/limpeza/alvejantes-e-amaciantes/agua-sanitaria?PS=50', 'https://www.superabc.com.br/limpeza/detergentes/liquidos?PS=50', 'https://www.superabc.com.br/limpeza/detergentes/em-po?PS=50', 'https://www.superabc.com.br/limpeza/detergentes/especiais?PS=50', 'https://www.superabc.com.br/limpeza/limpadores-e-purificadores/limpadores?PS=50', 'https://www.superabc.com.br/limpeza/limpadores-e-purificadores/purificadores?PS=50', 'https://www.superabc.com.br/limpeza/sabao-em-pasta-e-barra/barra?PS=50', 'https://www.superabc.com.br/limpeza/sabao-em-pasta-e-barra/pasta?PS=50', 'https://www.superabc.com.br/limpeza/sabao-em-pasta-e-barra/saponaceos?PS=50', 'https://www.superabc.com.br/limpeza/sabao-em-pasta-e-barra/institucional?PS=50', 'https://www.superabc.com.br/limpeza?PS=50', 'https://www.superabc.com.br/limpeza/limpadores-e-purificadores?PS=50', 'https://www.superabc.com.br/limpeza/alvejantes-e-amaciantes?PS=50', 'https://www.superabc.com.br/limpeza/detergentes?PS=50', 'https://www.superabc.com.br/limpeza/ceras-e-desinfetantes?PS=50', 'https://www.superabc.com.br/limpeza/sabao-em-pasta-e-barra?PS=50'], ['https://www.superabc.com.br/bazar/camping-e-esporte/camping?PS=50', 'https://www.superabc.com.br/bazar/camping-e-esporte/esporte?PS=50', 'https://www.superabc.com.br/bazar/cama-mesa-e-banho/cama-mesa-e-banho?PS=50', 'https://www.superabc.com.br/bazar/cama-mesa-e-banho/acessorios?PS=50', 'https://www.superabc.com.br/bazar/casa-e-cozinha/aluminio-e-ferro?PS=50', 'https://www.superabc.com.br/bazar/casa-e-cozinha/inox?PS=50', 'https://www.superabc.com.br/bazar/casa-e-cozinha/ceramica?PS=50', 'https://www.superabc.com.br/bazar/casa-e-cozinha/vidro-e-acrilico?PS=50', 'https://www.superabc.com.br/bazar/casa-e-cozinha/plastico?PS=50', 'https://www.superabc.com.br/bazar/casa-e-cozinha/utensilios?PS=50', 'https://www.superabc.com.br/bazar/casa-e-cozinha/utilidades?PS=50', 'https://www.superabc.com.br/bazar/roupas-e-calcados/roupas-e-acessorios?PS=50', 'https://www.superabc.com.br/bazar/casa-e-construcao/ferragens?PS=50', 'https://www.superabc.com.br/bazar/casa-e-construcao/acendimentos?PS=50', 'https://www.superabc.com.br/bazar/casa-e-construcao/eletricos?PS=50', 'https://www.superabc.com.br/bazar/casa-e-construcao/hidraulicos?PS=50', 'https://www.superabc.com.br/bazar/conveniencias/tabacaria?PS=50', 'https://www.superabc.com.br/bazar/eletroportateis/utilidades?PS=50', 'https://www.superabc.com.br/bazar/eletroportateis/portateis?PS=50', 'https://www.superabc.com.br/bazar/embalagens-e-descartaveis/embalagens-e-descartaveis?PS=50', 'https://www.superabc.com.br/bazar/embalagens-e-descartaveis/acessorios?PS=50', 'https://www.superabc.com.br/bazar/papelaria-e-festas/papelaria?PS=50', 'https://www.superabc.com.br/bazar/papelaria-e-festas/escola-e-escritorio?PS=50', 'https://www.superabc.com.br/bazar/papelaria-e-festas/festa?PS=50', 'https://www.superabc.com.br/bazar/automotivos/pneu?PS=50', 'https://www.superabc.com.br/bazar/automotivos/fluidos-e-lubrificantes?PS=50', 'https://www.superabc.com.br/bazar/automotivos/higienizacao-e-aromatizantes?PS=50', 'https://www.superabc.com.br/bazar/automotivos/acessorios?PS=50', 'https://www.superabc.com.br/bazar?PS=50', 'https://www.superabc.com.br/bazar/camping-e-esporte?PS=50', 'https://www.superabc.com.br/bazar/conveniencias?PS=50', 'https://www.superabc.com.br/bazar/roupas-e-calcados?PS=50', 'https://www.superabc.com.br/bazar/embalagens-e-descartaveis?PS=50', 'https://www.superabc.com.br/bazar/casa-e-cozinha?PS=50', 'https://www.superabc.com.br/bazar/automotivos?PS=50', 'https://www.superabc.com.br/bazar/cama-mesa-e-banho?PS=50', 'https://www.superabc.com.br/bazar/casa-e-construcao?PS=50', 'https://www.superabc.com.br/bazar/eletroportateis?PS=50', 'https://www.superabc.com.br/bazar/papelaria-e-festas?PS=50'], []]


In [112]:
x = [link for grupo in x for link in grupo if len(link)>0]

In [113]:
x

['https://www.superabc.com.br/mercearia/bomboniere/bombons-e-chocolates?PS=50',
 'https://www.superabc.com.br/mercearia/bomboniere/guloseimas?PS=50',
 'https://www.superabc.com.br/mercearia/bomboniere/doces-diversos?PS=50',
 'https://www.superabc.com.br/mercearia/funcionais-diet-e-light/integrais?PS=50',
 'https://www.superabc.com.br/mercearia/funcionais-diet-e-light/soja?PS=50',
 'https://www.superabc.com.br/mercearia/funcionais-diet-e-light/diet?PS=50',
 'https://www.superabc.com.br/mercearia/funcionais-diet-e-light/sem-gluten?PS=50',
 'https://www.superabc.com.br/mercearia/funcionais-diet-e-light/sem-lactose?PS=50',
 'https://www.superabc.com.br/mercearia/funcionais-diet-e-light/light?PS=50',
 'https://www.superabc.com.br/mercearia/funcionais-diet-e-light/organicos?PS=50',
 'https://www.superabc.com.br/mercearia/funcionais-diet-e-light/cereais?PS=50',
 'https://www.superabc.com.br/mercearia/matinais/complemento-alimentar?PS=50',
 'https://www.superabc.com.br/mercearia/matinais/achoc